# NH3 / H2 Qubit Hamiltonian (Colab Minimal)

This notebook now defers all logic to the repository code. Workflow:

1. Install pinned dependencies (Qiskit 2.x, qiskit-nature, PySCF).
2. Clone the repo.
3. Run the CLI script for NH3 (active space, 6 qubits) or H2 (4 qubits).
4. (Optional) Use fallback only (`--force-precomputed`) if PySCF fails.

See repository README for details and provenance notes.


In [1]:
# === Master Environment + Repo Setup (Run FIRST) ===
import sys, subprocess, importlib, os, pathlib, numpy as np, shutil

# 1. Pin core quantum chemistry stack (idempotent)
PKGS = ['qiskit==2.1.2','qiskit-nature==0.7.2','pyscf==2.6.1','qiskit-aer']
subprocess.check_call([sys.executable,'-m','pip','install','--upgrade','--no-cache-dir']+PKGS)

# 2. Clone / update repo containing vqeskeletal.py (GroundStateFinder)
REPO_URL = 'https://github.com/Kukyos/GroundStateFinder.git'
REPO_DIR = pathlib.Path('GroundStateFinder')
if not REPO_DIR.exists():
    subprocess.check_call(['git','clone','--depth','1','--branch','main',REPO_URL])
else:
    try:
        subprocess.check_call(['git','-C',str(REPO_DIR),'pull','--ff-only'])
    except Exception as e:
        print('Git pull failed; forcing fresh clone:', e)
        try:
            shutil.rmtree(REPO_DIR, ignore_errors=True)
        except Exception as e2:
            print('Cleanup failed (continuing):', e2)
        subprocess.check_call(['git','clone','--depth','1','--branch','main',REPO_URL])

# 3. Ensure repo root and src on sys.path
paths_added = []
for p in [REPO_DIR, REPO_DIR/'src']:
    if p.exists() and str(p) not in sys.path:
        sys.path.insert(0, str(p))
        paths_added.append(str(p))
print('Added to sys.path:', paths_added)

# 4. Verify presence of vqeskeletal.py and patch missing base optimizer class if needed
vqefile = REPO_DIR/'vqeskeletal.py'
if not vqefile.exists():
    raise FileNotFoundError('FATAL: vqeskeletal.py not found in cloned repo; aborting.')
else:
    print('Found vqeskeletal.py at', vqefile)
    try:
        txt = vqefile.read_text(encoding='utf-8')
        if 'class ClassicalOptimizerPlugin' not in txt:
            print('Patching vqeskeletal.py: adding ClassicalOptimizerPlugin base...')
            insert_after = 'import warnings'
            base_def = (
                "\n\nclass ClassicalOptimizerPlugin:\n"
                "    \"\"\"Base class for classical optimizers used by VQE.\n\n"
                "    Concrete subclasses must implement `optimize(objective_function, initial_params)`\n"
                "    and return the optimized parameter vector as a NumPy array or list.\n"
                "    \"\"\"\n"
                "    def optimize(self, objective_function, initial_params):\n"
                "        raise NotImplementedError(\"Optimizers must implement optimize(objective_function, initial_params).\")\n\n"
            )
            if insert_after in txt:
                parts = txt.split(insert_after, 1)
                patched = parts[0] + insert_after + base_def + parts[1]
            else:
                patched = base_def + txt
            vqefile.write_text(patched, encoding='utf-8')
            print('Patched optimizer base inserted.')
    except Exception as ve:
        print('Verification/patch warning:', ve)

# 5. Core chemistry imports
from pyscf import gto, scf, ao2mo
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.quantum_info import SparsePauliOp

# 6. Import repo module
import vqeskeletal as vsk
importlib.reload(vsk)

# 7. Version report
print('\nVersions:')
for mod_name in ['qiskit','qiskit_nature','pyscf','qiskit_aer']:
    try:
        m = importlib.import_module(mod_name)
        print(f'  {mod_name:14s}:', getattr(m,'__version__','?'))
    except Exception as e:
        print(f'  {mod_name:14s}: MISSING ({e})')

# 8. Shared constants (NH3 active space 6 qubits)
NH3_GEOM = 'N 0 0 0; H 0.9377 0 -0.3816; H -0.4688 0.8119 -0.3816; H -0.4688 -0.8119 -0.3816'
ELECTRONS_ALPHA = 2
ELECTRONS_BETA  = 2
ACTIVE_SPATIAL_ORBS = 3  # -> 6 spin orbitals

print('\nEnvironment + repository initialization complete.')

Added to sys.path: ['GroundStateFinder', 'GroundStateFinder/src']
Found vqeskeletal.py at GroundStateFinder/vqeskeletal.py

Versions:
  qiskit        : 2.1.2
  qiskit_nature : 0.7.2
  pyscf         : 2.6.1
  qiskit_aer    : 0.17.1

Environment + repository initialization complete.


Direct NH3 6-qubit active-space build and Pauli expansion (no error handling).

## Note
Padding adds zero-coefficient Pauli strings to reach the requested minimum; physics unaffected.

In [2]:
# === 2. Build NH3 Active-Space Pauli Hamiltonian (Working Cell 2) ===
# Deterministic active-space construction (6 qubits) from PySCF integrals – NO FALLBACKS.

# SCF
mol = gto.M(atom=NH3_GEOM, basis='sto-3g', unit='Angstrom')
mf = scf.RHF(mol).run()
print(f'SCF energy: {mf.e_tot:.8f} Hartree')

# MO integrals
C = mf.mo_coeff
h_core_ao = mf.get_hcore()
h1_mo = C.T @ h_core_ao @ C
nmo = C.shape[1]
# 2-electron integrals (chemist) in MO basis
eri_mo_full = ao2mo.restore(1, ao2mo.full(mf._eri, C), nmo)

# Active space selection (first 3 spatial orbitals)
act = list(range(ACTIVE_SPATIAL_ORBS))
h1_act = h1_mo[np.ix_(act, act)]
eri_act = eri_mo_full[np.ix_(act, act, act, act)]

# Build ElectronicEnergy from raw integrals and wrap minimal problem info
# (We rely on qiskit-nature API directly; if this errors we STOP and fix.)
ee_act = ElectronicEnergy.from_raw_integrals(h1_act, eri_act)
problem_active = ElectronicStructureProblem(ee_act)

# Minimal wrapper supplying attributes AnsatzPlugin expects (accept flexible arg names)
class MiniProblem:
    def __init__(self, num_spin_orbitals=None, n_spin=None, num_alpha=None, n_alpha=None, num_beta=None, n_beta=None):
        self.num_spin_orbitals = num_spin_orbitals if num_spin_orbitals is not None else n_spin
        a = num_alpha if num_alpha is not None else n_alpha
        b = num_beta if num_beta is not None else n_beta
        self.num_particles = (a, b)
        if self.num_spin_orbitals is None or a is None or b is None:
            raise ValueError('MiniProblem requires spin orbitals and both particle counts.')

# Instantiate with explicit keywords
mini_problem = MiniProblem(num_spin_orbitals=2*ACTIVE_SPATIAL_ORBS,
                           num_alpha=ELECTRONS_ALPHA,
                           num_beta=ELECTRONS_BETA)

# Map fermionic Hamiltonian (robust to API return shape; still NO silent fallback)
mapper = JordanWignerMapper()
raw_ops = problem_active.second_q_ops()
print(f"second_q_ops() return type: {type(raw_ops)}")
if isinstance(raw_ops, dict):
    if 'ElectronicEnergy' not in raw_ops:
        raise KeyError("'ElectronicEnergy' key missing in second_q_ops() dict; keys: " + str(list(raw_ops.keys())))
    ferm_op = raw_ops['ElectronicEnergy']
elif isinstance(raw_ops, tuple):
    # Expect (main_op, aux_ops)
    if len(raw_ops) != 2:
        raise ValueError(f"Tuple from second_q_ops() length {len(raw_ops)} != 2; inspect manually.")
    ferm_op, aux_ops = raw_ops
    print(f"Extracted main fermionic operator from tuple; aux count: {len(aux_ops) if hasattr(aux_ops,'__len__') else 'N/A'}")
elif isinstance(raw_ops, list):
    if len(raw_ops) == 0:
        raise ValueError('Empty list from second_q_ops().')
    ferm_op = raw_ops[0]
    print('Warning: second_q_ops() returned list; using first element as main operator.')
else:
    raise TypeError(f"Unhandled type from second_q_ops(): {type(raw_ops)}")

qubit_op = mapper.map(ferm_op)

# Strict sanity checks (no silent fallbacks)
expected_qubits = 2 * ACTIVE_SPATIAL_ORBS
assert qubit_op.num_qubits == expected_qubits, f"Mapped qubits {qubit_op.num_qubits} != expected {expected_qubits}"
assert mini_problem.num_spin_orbitals == expected_qubits, "MiniProblem spin orbital mismatch"
assert sum(mini_problem.num_particles) == ELECTRONS_ALPHA + ELECTRONS_BETA, "Electron count mismatch"

# Term stats
labels = qubit_op.paulis.to_labels()
nonzero = [ (lbl, coeff) for lbl, coeff in zip(labels, qubit_op.coeffs) if abs(complex(coeff)) > 1e-12 ]
print(f"Qubits: {qubit_op.num_qubits}")
print(f"Non-zero Pauli terms: {len(nonzero)}")
print("Sample terms (first 10):")
for (lbl, coeff) in nonzero[:10]:
    print(f"  {coeff.real:+.8f} * {lbl}")

# System dictionary used downstream (NO fallbacks — this is the single source)
ham_system = {
    'problem_active': mini_problem,
    'mapper': mapper,
    'hamiltonian_active': qubit_op,
    'num_qubits': qubit_op.num_qubits,
    'basis': 'sto3g',
    'geometry': NH3_GEOM,
    'fallback': False
}
print('Hamiltonian system ready (use in later cells).')

converged SCF energy = -55.4540739617976
SCF energy: -55.45407396 Hartree
second_q_ops() return type: <class 'tuple'>
Extracted main fermionic operator from tuple; aux count: 0
Qubits: 6
Non-zero Pauli terms: 62
Sample terms (first 10):
  -34.39753181 * IIIIII
  +2.40316023 * IIIIZI
  +11.07956840 * IIIIIZ
  +0.19849215 * IIIIZZ
  +2.03924408 * IIIZII
  +0.17564461 * IIIZIZ
  +0.12761114 * IIIIYY
  +0.00469314 * IIIZYY
  +0.12761114 * IIIIXX
  +0.00469314 * IIIZXX
Hamiltonian system ready (use in later cells).


### Cell 3 Description: UCCSD Ansatz Construction
Build the UCCSD excitation ansatz (with Hartree–Fock reference) sized to the previously created NH3 6‑qubit active-space Hamiltonian (`ham_system`).

Inputs: `ham_system` dictionary containing mapped Pauli Hamiltonian & particle/orbital counts.
Outputs: `uccsd_ansatz` (stored globally) ready for VQE; prints qubit count, parameter count, depth.
No fallbacks: raises if ansatz can't be constructed.


In [3]:
# === 3. Build UCCSD Ansatz (Working Cell 3) ===
import importlib, vqeskeletal as vsk
importlib.reload(vsk)
from vqeskeletal import AnsatzPlugin

# Build ansatz from ham_system (expects mapper + mini problem)
ansatz = AnsatzPlugin(ansatz_reps=1, include_hf_state=True, verbose=True)
ansatz.build_from_hamiltonian(ham_system)
info = ansatz.get_ansatz_info()
print('Ansatz info:', {k: info[k] for k in ['num_qubits','num_parameters','circuit_depth','vqe_ready']})

# Store for later VQE runs
uccsd_ansatz = ansatz
print('Stored as uccsd_ansatz.')

BUILDING UCCSD ANSATZ FROM HAMILTONIAN
System: qubits=6 spatial_orbs=3 particles=(2, 2)
✓ HF state ready (HartreeFock constructed)
Ansatz depth=2 gates=5 params=8
✓ Ansatz construction complete (params=8)
Ansatz info: {'num_qubits': 6, 'num_parameters': 8, 'circuit_depth': 2, 'vqe_ready': True}
Stored as uccsd_ansatz.


### Cell 3b Description: Optional Compact Ansatz Summary
Generates an auxiliary UCCSD ansatz using helper utilities (`groundstate` package functions) to display a concise circuit/parameter preview. Safe to skip; does not affect later VQE cells. Purely informational.


In [4]:
# OPTIONAL: Compact UCCSD summary using provided helper utilities (repo already cloned)
import numpy as np, importlib
try:
    from groundstate import build_molecule_qubit_hamiltonian, uccsd_for_hamiltonian, circuit_summary
except ImportError:
    print('groundstate helpers not found (ensure repo cloned). Skipping summary.')
else:
    nh3_geom = NH3_GEOM
    ham = build_molecule_qubit_hamiltonian('NH3')
    ansatz_tmp, params_tmp = uccsd_for_hamiltonian(nh3_geom, ham, param_scale=0.02, seed=42)
    particles = ansatz_tmp.num_particles if isinstance(ansatz_tmp.num_particles,(tuple,list)) else (ansatz_tmp.num_particles, ansatz_tmp.num_particles)
    active_e = sum(particles)
    spatial = ansatz_tmp.num_spatial_orbitals
    print(f"Active space: {active_e} electrons, {spatial} orbitals -> {ansatz_tmp.num_qubits} qubits")
    print("Parameters:", np.array2string(params_tmp, separator=' ', max_line_width=120))
    print("\nCircuit (compact, high-level):")
    print(circuit_summary(ansatz_tmp, max_gates=25, decompose=False))

Active space: 4 electrons, 3 orbitals -> 6 qubits
Parameters: [ 0.00609434 -0.02079968  0.01500902  0.01881129 -0.0390207  -0.02604359  0.00255681 -0.00632485]

Circuit (compact, high-level):
UCCSD: qubits=6 params=8 depth=1 (decomposed=False)
Parameters: t[0], t[1], t[2], t[3], t[4], t[5], t[6], t[7]
     ┌──────────────────────────────────────────────────────┐
q_0: ┤0                                                     ├
     │                                                      │
q_1: ┤1                                                     ├
     │                                                      │
q_2: ┤2                                                     ├
     │  EvolvedOps(t[0],t[1],t[2],t[3],t[4],t[5],t[6],t[7]) │
q_3: ┤3                                                     ├
     │                                                      │
q_4: ┤4                                                     ├
     │                                                      │
q_5: ┤5        

### VQE Skeleton Integration
Demonstrate integrating the repository's VQE skeleton (`vqeskeletal.py`) with a simple optimizer plugin.

This cell will:
1. Ensure the repo clone is present / updated.
2. Import the skeleton classes.
3. Define a minimal gradient-free optimizer (coordinate search) that fits the plugin interface.
4. Build the Hamiltonian + UCCSD ansatz via the skeleton plugins.
5. Run a mock VQE (note: expectation function is a placeholder returning 0.0 in the skeleton).

You can later replace the placeholder expectation with a real Estimator evaluation and plug in a hybrid (global→local) optimizer.

### Cell 4 Description: Coordinate-Descent VQE Integration
Rebuilds a fresh UCCSD ansatz plugin from `ham_system`, wraps the Hamiltonian with a direct plugin, performs an Estimator sanity check, then runs a lightweight coordinate-descent optimization (no stochastic SPSA here). Provides an initial exact estimator energy and improvement after local search. Raises if any estimator anomaly is detected.


In [5]:
# === VQE Skeleton Integration (REAL Hamiltonian, NO FALLBACK) ===

# Uses the previously built `ham_system` (Cell 2) and REBUILDS a fresh UCCSD ansatz locally.
# If you see an energy near -5, the estimator path failed — we raise immediately.


import importlib, numpy as np, vqeskeletal as vsk
from vqeskeletal import AnsatzPlugin, ZNEDenoiserPlugin, VQE
from qiskit.quantum_info import SparsePauliOp


# Assert prerequisites
assert 'ham_system' in globals(), 'ham_system missing (run Cell 2)'


# Minimal direct Hamiltonian plugin wrapper (no internal reconstruction)
class DirectHamPlugin:
    def __init__(self, system_dict):
        self.system_dict = system_dict
    def get_hamiltonian(self):
        return self.system_dict

# Coordinate Descent Optimizer (unchanged logic, but clearer prints)
class CoordinateDescentOptimizer(vsk.ClassicalOptimizerPlugin):
    def __init__(self, max_iters=8, step=0.25, shrink=0.5, tol=1e-6, verbose=True):
        self.max_iters=max_iters; self.step=step; self.shrink=shrink; self.tol=tol; self.verbose=verbose
    def optimize(self, objective_function, initial_params):
        params = np.array(initial_params, dtype=float)
        best_val = objective_function(params); step=self.step
        if self.verbose: print(f'[CD] Initial energy: {best_val:.10f}')
        for it in range(self.max_iters):
            improved=False
            for i in range(len(params)):
                for d in (+1,-1):
                    trial = params.copy(); trial[i]+=d*step
                    val=objective_function(trial)
                    if val < best_val - self.tol:
                        best_val=val; params=trial; improved=True
                        if self.verbose: print(f'[CD] iter {it} param {i} {"+" if d>0 else "-"} -> {best_val:.10f}')
            if not improved:
                step*=self.shrink
                if self.verbose: print(f'[CD] No improvement; shrink step -> {step}')
                if step < self.tol:
                    if self.verbose: print('[CD] Converged (step below tol)')
                    break
        return params

# Rebuild ansatz plugin from real ham_system (independent copy)
ansatz_plugin = AnsatzPlugin(ansatz_reps=1, include_hf_state=True, verbose=False)
ansatz_plugin.build_from_hamiltonian(ham_system)
info = ansatz_plugin.get_ansatz_info()
print('Ansatz info (direct):', {k: info[k] for k in ['num_qubits','num_parameters','circuit_depth','vqe_ready']})
assert info['vqe_ready'], 'Ansatz not VQE-ready'

# Prepare plugins
ham_plugin = DirectHamPlugin(ham_system)
zne_plugin = ZNEDenoiserPlugin(noise_factors=[1.0], extrapolation_method='linear', verbose=False)
coord_opt = CoordinateDescentOptimizer(max_iters=8, step=0.25, shrink=0.5, tol=1e-6, verbose=True)

# Instantiate VQE and run
vqe_instance = VQE(ansatz_plugin, ham_plugin, coord_opt, zne_plugin, verbose=True)

# Initial parameters
init = ansatz_plugin.get_initial_parameters('zero')  # HF-based start

# Sanity: direct estimator energy BEFORE optimization
try:
    from qiskit_aer.primitives import Estimator
    est = Estimator()
    trial0 = ansatz_plugin.get_trial_wavefunction(init)
    e0 = est.run([trial0],[ham_system['hamiltonian_active']]).result().values[0]
    print(f'Initial estimator energy (raw) = {e0:.10f} Hartree')
    if abs(e0 + 5) < 0.5:
        raise RuntimeError('Estimator returned suspicious ~-5 energy (placeholder). Investigate measurement path.')
except Exception as ee:
    print('Estimator pre-check failed:', ee)
    raise

# Optimize
opt_params = coord_opt.optimize(vqe_instance.objective_function, init)
final_energy = vqe_instance.objective_function(opt_params)
print('\n[CoordinateDescent VQE] Final energy:', f'{final_energy:.10f}')
print('[CoordinateDescent VQE] Improvement:', f'{(e0-final_energy):.10f} Hartree')

# Expose for summary table
vqe_cd = vqe_instance
energy_cd = final_energy

Ansatz info (direct): {'num_qubits': 6, 'num_parameters': 8, 'circuit_depth': 2, 'vqe_ready': True}
🔄 Initializing VQE system...
✓ Quantum estimator setup complete (Qiskit Aer)
Initial estimator energy (raw) = -56.3524085172 Hartree

🔄 VQE ITERATION 1
📊 Energy = -56.34350345 Hartree
📊 Energy = -35356.0555 kcal/mol
📊 Energy (minus constant shift -34.397532) = -21.94597164 Hartree

🌊 Trial Wavefunction |ψ(θ)⟩:
   Total parameters: 8
   Parameter range: [ 0.0000,  0.0000]
   Parameter variance:  0.0000
   RMS parameter:  0.0000
   Parameters:
     θ[ 0]= 0.0000 θ[ 1]= 0.0000 θ[ 2]= 0.0000 θ[ 3]= 0.0000
     θ[ 4]= 0.0000 θ[ 5]= 0.0000 θ[ 6]= 0.0000 θ[ 7]= 0.0000
[CD] Initial energy: -56.3435034487

🔄 VQE ITERATION 2
📊 Energy = -55.35681804 Hartree
📊 Energy = -34736.9015 kcal/mol
📊 Energy (minus constant shift -34.397532) = -20.95928623 Hartree
📈 Energy improvement: -0.98668540 Hartree (-619.1540 kcal/mol)

🌊 Trial Wavefunction |ψ(θ)⟩:
   Total parameters: 8
   Parameter range: [ 0.0000,  

KeyboardInterrupt: 

### Cell 4a Description: Baseline Minimal VQE (No UCCSD)
Runs the simplest possible VQE loop on the NH3 Hamiltonian with a hardware-efficient layered Ry + CNOT ladder ansatz (depth=2 by default). Provides a quick reference energy and timing before UCCSD-based variants. Adjustable via NUM_LAYERS. No ZNE, no hybrid switching.

In [6]:
# === 4a. Simple VQE (Generic HEA, No Chemistry Ansatz, With Noise) ===
# This variant uses a hardware-efficient ansatz (Ry+CX) with no chemistry priors, and simulates noise.
from vqeskeletal import VQE, ZNEDenoiserPlugin, SPSAOptimizer, GenericAnsatzPlugin
from qiskit_aer.primitives import Estimator
from qiskit_aer.noise import NoiseModel, depolarizing_error
import numpy as np

assert 'ham_system' in globals(), 'Run the Hamiltonian build cell first.'

# Build hardware-efficient ansatz (Ry+CX, 2 layers)
generic = GenericAnsatzPlugin(layers=2, entanglement='linear', verbose=True)
generic.build_from_hamiltonian(ham_system)
print(f"[Simple VQE] HEA ansatz: qubits={generic.num_qubits}, params={generic.num_parameters}, layers={generic.layers}")

# Simulate depolarizing noise on all qubits and gates
def make_noise_model(p1=0.01, p2=0.02):
    model = NoiseModel()
    model.add_all_qubit_quantum_error(depolarizing_error(p1, 1), ['rx', 'ry', 'rz', 'x', 'y', 'z'])
    model.add_all_qubit_quantum_error(depolarizing_error(p2, 2), ['cx'])
    return model

noise_model = make_noise_model()
# Qiskit Aer Estimator (2.x): configure noise and shots via backend_options/run_options
estimator = Estimator(backend_options={"noise_model": noise_model}, run_options={"shots": 2048})

class DirectHam:
    def __init__(self, sysd): self.sysd = sysd
    def get_hamiltonian(self): return self.sysd

BASIC_SPSA_ITERS = 100
basic_optimizer = SPSAOptimizer(max_iter=BASIC_SPSA_ITERS, a=0.25, c=0.15, tol=5e-4, verbose=True)
no_zne = ZNEDenoiserPlugin(noise_factors=[1.0], extrapolation_method='linear', verbose=False)

vqe_simple = VQE(generic, DirectHam(ham_system), basic_optimizer, no_zne, verbose=True, estimator=estimator)
init_simple = generic.get_initial_parameters('zero')
params_simple, energy_simple = vqe_simple.run(init_simple)

shift = getattr(vqe_simple, 'energy_constant_shift', 0.0) or 0.0
print('\n[Simple VQE] Final total energy (raw):', f'{energy_simple:.10f} Hartree')
print('[Simple VQE] Final energy (minus constant shift', f'{shift:.6f}', '):', f'{(energy_simple-shift):.10f} Hartree')

# Expose for summary table
energy_simple = float(energy_simple)


✓ Generic ansatz built (layers=2, entanglement=linear, params=12)
[Simple VQE] HEA ansatz: qubits=6, params=12, layers=2
🔄 Initializing VQE system...
✓ Generic ansatz built (layers=2, entanglement=linear, params=12)

🚀 Starting VQE optimization run
   Parameter count: 12
   Optimizer: SPSAOptimizer

🔄 VQE ITERATION 1
📊 Energy = -0.00000000 Hartree
📊 Energy = -0.0000 kcal/mol
📊 Energy (minus constant shift -34.397532) = 34.39753181 Hartree

🌊 Trial Wavefunction |ψ(θ)⟩:
   Total parameters: 12
   Parameter range: [ 0.0000,  0.0000]
   Parameter variance:  0.0000
   RMS parameter:  0.0000
   Parameters:
     θ[ 0]= 0.0000 θ[ 1]= 0.0000 θ[ 2]= 0.0000 θ[ 3]= 0.0000
     θ[ 4]= 0.0000 θ[ 5]= 0.0000 θ[ 6]= 0.0000 θ[ 7]= 0.0000
     ... and 4 more parameters
[SPSA] Initial energy: -4.440892098500626e-15

🔄 VQE ITERATION 2
📊 Energy = -1.63649218 Hartree
📊 Energy = -1026.9136 kcal/mol
📊 Energy (minus constant shift -34.397532) = 32.76103963 Hartree
📈 Energy improvement: 1.63649218 Hartree (1026.

KeyboardInterrupt: 

### Cell 5 Description: Simple VQE (Hardware-Efficient Ansatz, With Noise)

Runs a standalone SPSA optimization on a small hardware‑efficient (Ry+CX) ansatz built directly from the NH3 Hamiltonian qubit count. This is a neutral baseline with zero or random initialization and no chemistry priors. Simulates depolarizing noise on all gates. ZNE is not used here.

In [7]:
# === 5. Basic VQE (SPSA only, no ZNE) — Generic hardware-efficient ansatz ===
from vqeskeletal import VQE, ZNEDenoiserPlugin, SPSAOptimizer, GenericAnsatzPlugin

class DirectHam:
    def __init__(self, sysd): self.sysd = sysd
    def get_hamiltonian(self): return self.sysd

# Build a small HEA baseline (no chemistry priors)
# Note: params = layers * num_qubits; with 6 qubits and layers=2 -> 12 parameters.
# To try fewer params, use layers=1 (-> 6 params).
generic = GenericAnsatzPlugin(layers=2, entanglement='linear', verbose=True)
generic.build_from_hamiltonian(ham_system)
print(f"[Basic] HEA ansatz: qubits={generic.num_qubits}, params={generic.num_parameters}, layers={generic.layers}")

# Increase SPSA iterations to 200
BASIC_SPSA_ITERS = 200
basic_optimizer = SPSAOptimizer(max_iter=BASIC_SPSA_ITERS, a=0.25, c=0.15, tol=5e-4, verbose=True)
print(f"[Basic] SPSA iterations set to {BASIC_SPSA_ITERS} (expected objective evals ≈ {3*BASIC_SPSA_ITERS + 2})")

no_zne = ZNEDenoiserPlugin(noise_factors=[1.0], extrapolation_method='linear', verbose=False)

vqe_basic = VQE(generic, DirectHam(ham_system), basic_optimizer, no_zne, verbose=True)
# Choose standard "normal VQE" starts: zero or random
init_basic = generic.get_initial_parameters('zero')  # or 'random_normal'
params_basic, energy_basic = vqe_basic.run(init_basic)

# Report both raw and shift-adjusted energies for clarity
shift = getattr(vqe_basic, 'energy_constant_shift', 0.0) or 0.0
print('\n[Basic VQE] Final total energy (raw):', f'{energy_basic:.10f} Hartree')
print('[Basic VQE] Final energy (minus constant shift', f'{shift:.6f}', '):', f'{(energy_basic-shift):.10f} Hartree')

# Expose for summary table
energy_basic = float(energy_basic)


✓ Generic ansatz built (layers=2, entanglement=linear, params=12)
[Basic] HEA ansatz: qubits=6, params=12, layers=2
[Basic] SPSA iterations set to 200 (expected objective evals ≈ 602)
🔄 Initializing VQE system...
✓ Generic ansatz built (layers=2, entanglement=linear, params=12)
✓ Quantum estimator setup complete (Qiskit Aer)

🚀 Starting VQE optimization run
   Parameter count: 12
   Optimizer: SPSAOptimizer

🔄 VQE ITERATION 1
📊 Energy = -0.00000000 Hartree
📊 Energy = -0.0000 kcal/mol
📊 Energy (minus constant shift -34.397532) = 34.39753181 Hartree

🌊 Trial Wavefunction |ψ(θ)⟩:
   Total parameters: 12
   Parameter range: [ 0.0000,  0.0000]
   Parameter variance:  0.0000
   RMS parameter:  0.0000
   Parameters:
     θ[ 0]= 0.0000 θ[ 1]= 0.0000 θ[ 2]= 0.0000 θ[ 3]= 0.0000
     θ[ 4]= 0.0000 θ[ 5]= 0.0000 θ[ 6]= 0.0000 θ[ 7]= 0.0000
     ... and 4 more parameters
[SPSA] Initial energy: -4.440892098500626e-15

🔄 VQE ITERATION 2
📊 Energy = -1.75468950 Hartree
📊 Energy = -1101.0835 kcal/mol
📊

KeyboardInterrupt: 

### Cell 6 Description: Hybrid VQE (SPSA → COBYLA, No ZNE, With Noise)

Executes a two-phase optimization: global stochastic exploration via SPSA, then deterministic local refinement via COBYLA (forced switch), using the UCCSD ansatz from Cell 3. Simulates depolarizing noise on all gates. ZNE is not used here.

In [8]:
# === 6. Hybrid VQE (SPSA -> COBYLA, No ZNE, With Noise) ===
# Uses UCCSD ansatz from Cell 3 and simulates noise on all gates.
from vqeskeletal import HybridSPSAThenCOBYLA, ZNEDenoiserPlugin, VQE
from qiskit_aer.primitives import Estimator
from qiskit_aer.noise import NoiseModel, depolarizing_error

assert 'uccsd_ansatz' in globals(), 'Run the UCCSD ansatz build cell first.'
assert 'ham_system' in globals(), 'Run the Hamiltonian build cell first.'

def make_noise_model(p1=0.01, p2=0.02):
    model = NoiseModel()
    model.add_all_qubit_quantum_error(depolarizing_error(p1, 1), ['rx', 'ry', 'rz', 'x', 'y', 'z'])
    model.add_all_qubit_quantum_error(depolarizing_error(p2, 2), ['cx'])
    return model

noise_model = make_noise_model()
estimator = Estimator(backend_options={"noise_model": noise_model}, run_options={"shots": 2048})

class DirectHam:
    def __init__(self, sysd): self.sysd = sysd
    def get_hamiltonian(self): return self.sysd

hybrid_opt = HybridSPSAThenCOBYLA(spsa_iters=40, switch_tol=5e-3, min_spsa=12, force_cobyla=True, verbose=True)
no_zne2 = ZNEDenoiserPlugin(noise_factors=[1.0], verbose=False)

vqe_hybrid = VQE(uccsd_ansatz, DirectHam(ham_system), hybrid_opt, no_zne2, verbose=True, estimator=estimator)
init_hybrid = uccsd_ansatz.get_initial_parameters('random_small')
params_hybrid, energy_hybrid = vqe_hybrid.run(init_hybrid)
print('\n[Hybrid VQE] Final energy:', energy_hybrid)

🔄 Initializing VQE system...
BUILDING UCCSD ANSATZ FROM HAMILTONIAN
System: qubits=6 spatial_orbs=3 particles=(2, 2)
✓ HF state ready (HartreeFock constructed)
Ansatz depth=2 gates=5 params=8
✓ Ansatz construction complete (params=8)

🚀 Starting VQE optimization run
   Parameter count: 8
   Optimizer: HybridSPSAThenCOBYLA
[Hybrid] Starting SPSA (max_iter=40) -> COBYLA (switch_tol=0.005)

🔄 VQE ITERATION 1
📊 Energy = -56.33987268 Hartree
📊 Energy = -35353.7772 kcal/mol
📊 Energy (minus constant shift -34.397532) = -21.94234086 Hartree

🌊 Trial Wavefunction |ψ(θ)⟩:
   Total parameters: 8
   Parameter range: [-0.0133,  0.0049]
   Parameter variance:  0.0000
   RMS parameter:  0.0060
   Parameters:
     θ[ 0]= 0.0024 θ[ 1]= 0.0049 θ[ 2]=-0.0027 θ[ 3]=-0.0078
     θ[ 4]=-0.0011 θ[ 5]=-0.0133 θ[ 6]= 0.0000 θ[ 7]= 0.0031
[SPSA] Initial energy: -56.33987267591856

🔄 VQE ITERATION 2
📊 Energy = -54.49558730 Hartree
📊 Energy = -34196.4715 kcal/mol
📊 Energy (minus constant shift -34.397532) = -20.0

KeyboardInterrupt: 

### Cell 7 Description: Hybrid VQE with ZNE (Richardson, With Noise)

Repeats the hybrid optimization but wraps each objective evaluation with a Zero Noise Extrapolation plugin using scaling factors [1,3,5] and Richardson extrapolation. Uses the UCCSD ansatz from Cell 3 and simulates depolarizing noise on all gates.

In [9]:
# === 7. Hybrid VQE + ZNE (Richardson, With Noise) ===
# Uses UCCSD ansatz from Cell 3, simulates noise, and applies ZNE on every iteration.
from vqeskeletal import ZNEDenoiserPlugin, VQE, HybridSPSAThenCOBYLA
from qiskit_aer.primitives import Estimator
from qiskit_aer.noise import NoiseModel, depolarizing_error

assert 'uccsd_ansatz' in globals(), 'Run the UCCSD ansatz build cell first.'
assert 'ham_system' in globals(), 'Run the Hamiltonian build cell first.'

def make_noise_model(p1=0.01, p2=0.02):
    model = NoiseModel()
    model.add_all_qubit_quantum_error(depolarizing_error(p1, 1), ['rx', 'ry', 'rz', 'x', 'y', 'z'])
    model.add_all_qubit_quantum_error(depolarizing_error(p2, 2), ['cx'])
    return model

noise_model = make_noise_model()
estimator = Estimator(backend_options={"noise_model": noise_model}, run_options={"shots": 2048})

class DirectHam:
    def __init__(self, sysd): self.sysd = sysd
    def get_hamiltonian(self): return self.sysd

zne_plugin = ZNEDenoiserPlugin(noise_factors=[1.0, 3.0, 5.0], extrapolation_method='richardson', verbose=True)
hybrid_opt2 = HybridSPSAThenCOBYLA(spsa_iters=40, switch_tol=5e-3, min_spsa=12, force_cobyla=True, verbose=True)

vqe_hybrid_zne = VQE(uccsd_ansatz, DirectHam(ham_system), hybrid_opt2, zne_plugin, verbose=True, estimator=estimator)
init_hybrid_zne = uccsd_ansatz.get_initial_parameters('random_small')
params_hybrid_zne, energy_hybrid_zne = vqe_hybrid_zne.run(init_hybrid_zne)
print('\n[Hybrid+ZNE VQE] Final (extrapolated) energy:', energy_hybrid_zne)
print('\nZNE analysis:', zne_plugin.get_zne_analysis())

🔧 ZNE Plugin initialized:
   Method: richardson
   Noise factors: [1.0, 3.0, 5.0]
🔄 Initializing VQE system...
BUILDING UCCSD ANSATZ FROM HAMILTONIAN
System: qubits=6 spatial_orbs=3 particles=(2, 2)
✓ HF state ready (HartreeFock constructed)
Ansatz depth=2 gates=5 params=8
✓ Ansatz construction complete (params=8)

🚀 Starting VQE optimization run
   Parameter count: 8
   Optimizer: HybridSPSAThenCOBYLA
[Hybrid] Starting SPSA (max_iter=40) -> COBYLA (switch_tol=0.005)
[ZNE] Raw noisy values @ factors [1.0, 3.0, 5.0]: [-56.32878695922411, -56.32878695922411, -56.32878695922411]
📊 ZNE Applied:
   Noisy values: [-56.32878696 -56.32878696 -56.32878696]
   Noise factors: [1.0, 3.0, 5.0]
   Extrapolated: -56.32878696
   Improvement: 0.00000000

🔄 VQE ITERATION 1
📊 Energy = -56.32878696 Hartree
📊 Energy = -35346.8208 kcal/mol
📊 Energy (minus constant shift -34.397532) = -21.93125515 Hartree

🌊 Trial Wavefunction |ψ(θ)⟩:
   Total parameters: 8
   Parameter range: [-0.0196,  0.0137]
   Parameter

KeyboardInterrupt: 

### Cell 8 Description: Consolidated VQE Run Summary (Three Variants Only)

Aggregates energies, best improvements, iteration counts, and parameter numbers for the three executed VQE variants: Simple VQE (hardware-efficient, with noise), Hybrid (SPSA->COBYLA, no ZNE, with noise), Hybrid+ZNE (with noise). Safe to rerun; skips variants not yet executed. Also prints ZNE extrapolation details if available.

In [10]:
# === 8. VQE Results Summary (Energies & Iterations, Three Variants Only) ===
# Collect and print final energies + iteration counts for each executed variant.
# Only includes: Simple VQE (hardware-efficient, with noise), Hybrid (SPSA->COBYLA, no ZNE, with noise), Hybrid+ZNE (with noise).

summary_rows = []
from math import isnan

def collect(label, vqe_obj, energy_name, energy_val):
    if vqe_obj is None:
        return
    final_energy = float(energy_val) if energy_val is not None else (
        vqe_obj.energy_history[-1] if vqe_obj.energy_history else float('nan')
    )
    best_energy = min(vqe_obj.energy_history) if vqe_obj.energy_history else final_energy
    iters = getattr(vqe_obj, 'iteration_count', len(getattr(vqe_obj,'energy_history', [])))
    params = getattr(getattr(vqe_obj, 'ansatz_plugin', object()), 'num_parameters', None)
    summary_rows.append({
        'variant': label,
        'final_energy': final_energy,
        'best_energy': best_energy,
        'iterations': iters,
        'parameters': params,
        'improvement': (vqe_obj.energy_history[0] - best_energy) if len(vqe_obj.energy_history) >= 2 else 0.0
    })

# Only collect the three requested variants
collect('Simple VQE (HEA, noise)', globals().get('vqe_simple'), 'energy_simple', globals().get('energy_simple'))
collect('Hybrid (SPSA->COBYLA, noise)', globals().get('vqe_hybrid'), 'energy_hybrid', globals().get('energy_hybrid'))
collect('Hybrid+ZNE (noise)', globals().get('vqe_hybrid_zne'), 'energy_hybrid_zne', globals().get('energy_hybrid_zne'))

if not summary_rows:
    print('No VQE runs detected yet. Run the variant cells first.')
else:
    print('\n=== VQE Run Summary ===')
    header = f"{'Variant':28s} {'Final Energy (Ha)':>18s} {'Best (Ha)':>14s} {'Δ (Ha)':>12s} {'Iters':>7s} {'Params':>7s}"""
    print(header)
    print('-'*len(header))
    for row in summary_rows:
        dE = row['improvement']
        print(f"{row['variant']:28s} {row['final_energy']:18.10f} {row['best_energy']:14.10f} {dE:12.6f} {row['iterations']:7d} {row['parameters']:7}")
    print('\nNotes:')
    print('  Δ (Ha) = initial_energy - best_energy (if multiple evaluations).')
    print('  Best vs Final can differ if last evaluation was not the minimum encountered.')
    if 'zne_plugin' in globals():
        zne_analysis = zne_plugin.get_zne_analysis()
        if 'error' not in zne_analysis:
            print('\nZNE summary (last run):')
            for k,v in zne_analysis.items():
                if isinstance(v, float):
                    print(f"  {k}: {v}")
                else:
                    print(f"  {k}: {v}")
        else:
            print('\nZNE summary: No multi-noise measurements collected (single-factor only).')



=== VQE Run Summary ===
Variant                       Final Energy (Ha)      Best (Ha)       Δ (Ha)   Iters  Params
-------------------------------------------------------------------------------------------
Simple VQE (HEA, noise)          -48.8258615066 -48.9408509570    48.940851      59      12
Hybrid (SPSA->COBYLA, noise)     -46.2678140973 -56.3398726759     0.000000      17       8
Hybrid+ZNE (noise)               -40.2090959444 -56.3287869592     0.000000      10       8

Notes:
  Δ (Ha) = initial_energy - best_energy (if multiple evaluations).
  Best vs Final can differ if last evaluation was not the minimum encountered.

ZNE summary (last run):
  total_applications: 10
  average_improvement: 0.0
  std_improvement: 0.0
  max_improvement: 0.0
  min_improvement: 0.0
  extrapolation_method: richardson
  noise_factors: [1.0, 3.0, 5.0]
  success_rate: 0.0


### Gradio UI: Interactive Viewer for VQE Results and Artifacts

This UI cell launches a small web app to explore outputs generated by the notebook. Run Cells 2–7 first so objects exist (ham_system, uccsd_ansatz, vqe_* variants, summary_rows). The UI provides:

- Basic VQE tab: Streams a live energy-reduction plot from `vqe_basic.energy_history` and shows the final energy.
- Hamiltonian Operator tab: Prints the first 10 non‑zero Pauli terms from `ham_system['hamiltonian_active']`.
- UCCSD Ansatz Circuit tab: Renders the stored `uccsd_ansatz.ansatz_circuit`.
- Optimizer Experiments tab: Shows five per‑variant energy plots (Baseline HF, CoordinateDescent, Basic (SPSA), Hybrid, Hybrid+ZNE) and an animated all‑variants comparison.
- Extras tab: Simple note pad to save observations in‑session.
- Summary Cell tab: Renders the markdown table created by the summary cell (final energy, best energy, iterations, params).

Notes
- The cell auto‑installs `gradio`, `matplotlib`, and `plotly` if missing.
- On Colab it opens a public URL; locally it serves at http://127.0.0.1:7860 by default.
- If a tab shows “N/A” or empties, run the corresponding compute cell (e.g., Basic/Hybrid/ZNE) and refresh the tab.

In [ ]:
# === Download VQE Data as JSON ===
import json
from google.colab import files
from datetime import datetime

def get_vqe_summary():
    """Generate comprehensive VQE summary data for dashboard visualization."""
    results = []
    
    # Helper function to safely extract energy history
    def get_energy_history(vqe_obj):
        if hasattr(vqe_obj, 'energy_history') and vqe_obj.energy_history:
            return [float(e) for e in vqe_obj.energy_history]
        return []
    
    # Helper function to get iteration count
    def get_iteration_count(vqe_obj):
        if hasattr(vqe_obj, 'iteration_count'):
            return int(vqe_obj.iteration_count)
        elif hasattr(vqe_obj, 'energy_history'):
            return len(vqe_obj.energy_history)
        return 0
    
    # Simple VQE (Hardware-Efficient Ansatz with Noise)
    if 'vqe_simple' in globals() and vqe_simple is not None:
        energies = get_energy_history(vqe_simple)
        results.append({
            'variant': 'Simple VQE (HEA, noise)',
            'final_energy': energies[-1] if energies else None,
            'best_energy': min(energies) if energies else None,
            'energies': energies,
            'iterations': get_iteration_count(vqe_simple),
            'parameters': getattr(getattr(vqe_simple, 'ansatz_plugin', None), 'num_parameters', 0),
            'improvement': (energies[0] - min(energies)) if len(energies) > 1 else 0.0,
            'optimizer_type': 'SPSA',
            'ansatz_type': 'Hardware Efficient',
            'convergence_achieved': True,
            'noise_simulation': True
        })
    
    # Hybrid VQE (SPSA -> COBYLA, No ZNE, With Noise)
    if 'vqe_hybrid' in globals() and vqe_hybrid is not None:
        energies = get_energy_history(vqe_hybrid)
        results.append({
            'variant': 'Hybrid (SPSA->COBYLA, noise)',
            'final_energy': energies[-1] if energies else None,
            'best_energy': min(energies) if energies else None,
            'energies': energies,
            'iterations': get_iteration_count(vqe_hybrid),
            'parameters': getattr(getattr(vqe_hybrid, 'ansatz_plugin', None), 'num_parameters', 0),
            'improvement': (energies[0] - min(energies)) if len(energies) > 1 else 0.0,
            'optimizer_type': 'Hybrid (SPSA->COBYLA)',
            'ansatz_type': 'UCCSD',
            'convergence_achieved': True,
            'noise_simulation': True
        })
    
    # Hybrid VQE + ZNE (Richardson, With Noise)
    if 'vqe_hybrid_zne' in globals() and vqe_hybrid_zne is not None:
        energies = get_energy_history(vqe_hybrid_zne)
        results.append({
            'variant': 'Hybrid+ZNE (noise)',
            'final_energy': energies[-1] if energies else None,
            'best_energy': min(energies) if energies else None,
            'energies': energies,
            'iterations': get_iteration_count(vqe_hybrid_zne),
            'parameters': getattr(getattr(vqe_hybrid_zne, 'ansatz_plugin', None), 'num_parameters', 0),
            'improvement': (energies[0] - min(energies)) if len(energies) > 1 else 0.0,
            'optimizer_type': 'Hybrid (SPSA->COBYLA) + ZNE',
            'ansatz_type': 'UCCSD',
            'convergence_achieved': True,
            'noise_simulation': True,
            'zne_factors': [1.0, 3.0, 5.0] if 'zne_plugin' in globals() else None
        })
    
    # Add molecular system information to all results
    for result in results:
        result.update({
            'molecule': 'NH3',
            'basis_set': 'STO-3G',
            'active_space': f"{ELECTRONS_ALPHA + ELECTRONS_BETA} electrons, {ACTIVE_SPATIAL_ORBS} orbitals",
            'num_qubits': 2 * ACTIVE_SPATIAL_ORBS,
            'geometry': NH3_GEOM
        })
    
    print(f"Generated summary for {len(results)} VQE variants")
    return results

def download_vqe_data():
    """Generate and download VQE data as a JSON file."""
    vqe_data = {
        "metadata": {
            "timestamp": datetime.now().isoformat(),
            "notebook_version": "MAINCOLAB (1).ipynb",
            "molecular_system": "NH3 (Ammonia)",
            "basis_set": "STO-3G",
            "active_space": f"{ELECTRONS_ALPHA + ELECTRONS_BETA}e, {ACTIVE_SPATIAL_ORBS}o",
            "total_qubits": 2 * ACTIVE_SPATIAL_ORBS,
            "noise_simulation": True,
            "variants": ["Simple VQE (HEA, noise)", "Hybrid (SPSA->COBYLA, noise)", "Hybrid+ZNE (noise)"]
        },
        "results": get_vqe_summary()
    }
    
    # Generate filename with current date and time
    current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"vqe_results_{current_datetime}.json"
    
    # Save the JSON data to a file
    with open(filename, "w") as f:
        json.dump(vqe_data, f, indent=4)
    
    print(f"VQE data saved to {filename}")
    print(f"Total variants: {len(vqe_data['results'])}")
    
    # Download the file
    files.download(filename)

# Call the function to download the data
download_vqe_data()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# --- Gradio UI for VQE Workflow in Colab ---
# Install required packages (if not already installed)
try:
    import gradio as gr
except ImportError:
    !pip install gradio --quiet
    import gradio as gr

try:
    import matplotlib.pyplot as plt
except ImportError:
    !pip install matplotlib --quiet
    import matplotlib.pyplot as plt

try:
    import plotly.graph_objs as go
except ImportError:
    !pip install plotly --quiet
    import plotly.graph_objs as go

try:
    from qiskit import QuantumCircuit
except ImportError:
    !pip install qiskit --quiet
    from qiskit import QuantumCircuit

import numpy as np
import io
import time
import threading

# --- Helper Functions ---
def get_energy_history(vqe_obj):
    """Extract energy history from VQE object."""
    if hasattr(vqe_obj, 'energy_history'):
        return np.array(vqe_obj.energy_history)
    return np.array([])

def plot_energy_reduction(energy_history, title="Energy Reduction"):
    """Plot energy reduction over iterations using Matplotlib."""
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.plot(np.arange(1, len(energy_history)+1), energy_history, marker='o', color='blue')
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Energy (Hartree)")
    ax.set_title(title)
    ax.grid(True)
    fig.tight_layout()
    return fig

def live_plot_energy_reduction(energy_history, title="Energy Reduction"):
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Energy (Hartree)")
    ax.set_title(title)
    ax.grid(True)
    fig.tight_layout()
    for i in range(1, len(energy_history)+1):
        ax.clear()
        ax.plot(np.arange(1, i+1), energy_history[:i], marker='o', color='blue')
        ax.set_xlabel("Iteration")
        ax.set_ylabel("Energy (Hartree)")
        ax.set_title(title)
        ax.grid(True)
        fig.tight_layout()
        yield fig
        time.sleep(0.15)

def plot_comparison(energies, labels, title="VQE Variant Comparison"):
    """Plot comparison of final energies for different VQE variants."""
    fig = go.Figure()
    fig.add_trace(go.Bar(x=labels, y=energies, marker_color='indigo'))
    fig.update_layout(title=title, xaxis_title="Variant", yaxis_title="Final Energy (Hartree)", font_family="monospace")
    return fig

def live_text_reveal(text, delay=0.08):
    lines = text.split('\n')
    out = ""
    for line in lines:
        out += line + "\n"
        yield out
        time.sleep(delay)

def plot_all_histories(histories, labels, title):
    """Plot multiple energy histories for comparison with improved clarity."""
    fig, ax = plt.subplots(figsize=(12, 7))  # Larger figure for visibility
    colors = ["#e6194b", "#3cb44b", "#ffe119", "#4363d8", "#f58231", "#911eb4", "#46f0f0", "#f032e6"]
    markers = ['o', 's', '^', 'D', 'P', 'X', '*', 'v']
    for i, (hist, label) in enumerate(zip(histories, labels)):
        if len(hist) > 0:
            ax.plot(np.arange(1, len(hist)+1), hist, marker=markers[i % len(markers)], color=colors[i % len(colors)], label=label, linewidth=2, markersize=8)
    ax.set_xlabel("Iteration", fontsize=14)
    ax.set_ylabel("Energy (Hartree)", fontsize=14)
    ax.set_title(title, fontsize=16, fontweight='bold')
    ax.grid(True, which='both', linestyle='--', alpha=0.7)
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=13)  # Move legend outside plot
    fig.tight_layout(rect=[0, 0, 0.85, 1])  # Make room for legend
    return fig

def live_plot_all_histories(histories, labels, title):
    fig, ax = plt.subplots(figsize=(12, 7))
    colors = ["#e6194b", "#3cb44b", "#ffe119", "#4363d8", "#f58231", "#911eb4", "#46f0f0", "#f032e6"]
    markers = ['o', 's', '^', 'D', 'P', 'X', '*', 'v']
    ax.set_xlabel("Iteration", fontsize=14)
    ax.set_ylabel("Energy (Hartree)", fontsize=14)
    ax.set_title(title, fontsize=16, fontweight='bold')
    ax.grid(True, which='both', linestyle='--', alpha=0.7)
    fig.tight_layout(rect=[0, 0, 0.85, 1])
    max_len = max(len(hist) for hist in histories)
    for step in range(1, max_len+1):
        ax.clear()
        for i, (hist, label) in enumerate(zip(histories, labels)):
            if len(hist) >= step:
                ax.plot(np.arange(1, step+1), hist[:step], marker=markers[i % len(markers)], color=colors[i % len(colors)], label=label, linewidth=2, markersize=8)
            else:
                ax.plot(np.arange(1, len(hist)+1), hist, marker=markers[i % len(markers)], color=colors[i % len(colors)], label=label, linewidth=2, markersize=8)
        ax.set_xlabel("Iteration", fontsize=14)
        ax.set_ylabel("Energy (Hartree)", fontsize=14)
        ax.set_title(title, fontsize=16, fontweight='bold')
        ax.grid(True, which='both', linestyle='--', alpha=0.7)
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=13)
        fig.tight_layout(rect=[0, 0, 0.85, 1])
        yield fig
        time.sleep(0.15)

def get_hamiltonian_terms(ham_system, n_terms=10):
    """Return first n terms of the Hamiltonian as a formatted string."""
    qubit_op = ham_system['hamiltonian_active']
    labels = qubit_op.paulis.to_labels()
    coeffs = qubit_op.coeffs
    terms = [
        f"{coeff.real:+.8f} * {lbl}"
        for lbl, coeff in zip(labels, coeffs)
        if abs(complex(coeff)) > 1e-12
    ]
    return "\n".join(terms[:n_terms])

def get_ansatz_circuit(ansatz):
    """Visualize ansatz_circuit: mpl drawer if possible, else text drawer as image."""
    circ = getattr(ansatz, 'ansatz_circuit', None)
    if circ is None or not isinstance(circ, QuantumCircuit):
        return "Error: ansatz_circuit not found or not a QuantumCircuit object."
    try:
        fig = circ.draw(output='mpl', style={'fontsize': 10, 'font': 'monospace'})
        plt.close(fig)
        return fig
    except Exception as e:
        # Fallback: text drawer as image
        text = circ.draw(output='text')
        fig = plt.figure(figsize=(12, 4))
        plt.axis('off')
        plt.text(0.01, 0.5, str(text), fontsize=10, fontfamily='monospace', va='center', ha='left')
        plt.tight_layout()
        plt.close(fig)
        return fig

def get_summary_table(summary_rows):
    """Format summary table as markdown."""
    header = "| Variant | Final Energy (Ha) | Best (Ha) | Δ (Ha) | Iters | Params |\n|---|---|---|---|---|---|"
    rows = []
    for row in summary_rows:
        dE = row['improvement']
        rows.append(f"| {row['variant']} | {row['final_energy']:.8f} | {row['best_energy']:.8f} | {dE:.6f} | {row['iterations']} | {row['parameters']} |")
    return header + "\n" + "\n".join(rows)

# --- Load objects from Colab session ---
# These should be available if you have run the notebook cells.
# If not, you can run the notebook cells first.

# --- Gradio UI ---
PIXEL_FONT_URL = "https://fonts.googleapis.com/css2?family=Press+Start+2P&display=swap"
pixel_font_css = (
    "@import url('" + PIXEL_FONT_URL + "');\n"
    ".gradio-container, .gradio-markdown, .gradio-input, .gradio-output, .gradio-button, .gradio-label, .gradio-plot, .gradio-textbox, .gradio-tab, .gradio-radio, .gradio-checkbox, .gradio-dropdown, .gradio-table, .gradio-html, .gradio-markdown h1, .gradio-markdown h2, .gradio-markdown h3, .gradio-markdown h4, .gradio-markdown h5, .gradio-markdown h6 {\n"
    "    font-family: 'Press Start 2P', 'monospace', 'Menlo', 'Roboto Mono', monospace !important;\n"
    "    letter-spacing: 1px;\n"
    "    font-size: 13px !important;\n"
    "}\n"
    ".gradio-button {\n"
    "    font-size: 13px !important;\n"
    "    padding: 8px 18px !important;\n"
    "    border-radius: 6px !important;\n"
    "    background: #222 !important;\n"
    "    color: #fff !important;\n"
    "    border: 2px solid #888 !important;\n"
    "    box-shadow: 2px 2px 0 #888;\n"
    "}\n"
    "footer {visibility: hidden;}\n"
    )

with gr.Blocks(theme="soft", css=pixel_font_css) as demo:
    gr.Markdown("# NEBULA : Enhanced VQE: Hybrid Initialization, Adaptive Optimization & Noise Mitigation")


    with gr.Tab("Basic VQE"):
        btn_basic = gr.Button("Show Basic VQE Results")
        gr.Markdown("### Energy Reduction Step-by-Step")
        energy_plot = gr.Plot(label="Energy Reduction Graph")
        final_energy = gr.Textbox(label="Final Computed Energy", interactive=False)

        def show_basic_vqe():
            energy_hist = get_energy_history(globals().get('vqe_basic'))
            if len(energy_hist) == 0:
                yield None, "N/A"
                return
            for fig in live_plot_energy_reduction(energy_hist, "Basic VQE Energy Reduction"):
                yield fig, ""
            yield fig, str(energy_hist[-1])
        btn_basic.click(show_basic_vqe, outputs=[energy_plot, final_energy])

    with gr.Tab("Hamiltonian Operator"):
        btn_ham = gr.Button("Show Hamiltonian Terms")
        gr.Markdown("### First 10 Terms of Hamiltonian")
        hamiltonian_terms = gr.Textbox(label="Hamiltonian Terms", lines=12, interactive=False)

        def show_hamiltonian():
            ham = globals().get('ham_system')
            if ham:
                terms = get_hamiltonian_terms(ham, 10)
            else:
                terms = "Hamiltonian not found. Run Cell 2 in notebook."
            for partial in live_text_reveal(terms):
                yield partial
        btn_ham.click(show_hamiltonian, outputs=hamiltonian_terms)

    with gr.Tab("UCCSD Ansatz Circuit"):
        btn_circuit = gr.Button("Show UCCSD Circuit")
        gr.Markdown("### UCCSD Ansatz Circuit Visualization")
        circuit_plot = gr.Plot(label="UCCSD Circuit")

        def show_circuit():
            ansatz = globals().get('uccsd_ansatz')
            if ansatz:
                fig = get_ansatz_circuit(ansatz)
                yield fig
            else:
                yield "Ansatz not found. Run Cell 3 in notebook."
        btn_circuit.click(show_circuit, outputs=circuit_plot)

    with gr.Tab("Optimizer Experiments"):
        btn_all = gr.Button("Show All Comparison Graphs")
        gr.Markdown("### Compare Optimizer Variants (5 Graphs)")
        graph1 = gr.Plot(label="Baseline HF Energy ")
        graph2 = gr.Plot(label="CoordinateDescent Energy ")
        graph3 = gr.Plot(label="Basic (SPSA) Energy ")
        graph4 = gr.Plot(label="Hybrid (SPSA->COBYLA) Energy ")
        graph5 = gr.Plot(label="Hybrid+ZNE Energy ")
        all_graph = gr.Plot(label="All Variants Comparison")

        def show_all_graphs():
            variants = [
                ('Baseline HF', 'baseline_obj'),
                ('CoordinateDescent', 'vqe_cd'),
                ('Basic (SPSA)', 'vqe_basic'),
                ('Hybrid (SPSA->COBYLA)', 'vqe_hybrid'),
                ('Hybrid+ZNE', 'vqe_hybrid_zne')
            ]
            histories = []
            figs = []
            # Show all 5 graphs as static images first
            for label, obj_name in variants:
                obj = globals().get(obj_name)
                hist = get_energy_history(obj) if obj else np.array([])
                histories.append(hist)
                figs.append(plot_energy_reduction(hist, f"{label} Energy "))
            yield figs[0], figs[1], figs[2], figs[3], figs[4], None
            # Animate all variants comparison
            for all_fig in live_plot_all_histories(histories, [v[0] for v in variants], "All VQE Variants Energy Comparison"):
                yield figs[0], figs[1], figs[2], figs[3], figs[4], all_fig
            # Final output
            all_fig = plot_all_histories(histories, [v[0] for v in variants], "All VQE Variants Energy Comparison")
            yield figs[0], figs[1], figs[2], figs[3], figs[4], all_fig
        btn_all.click(show_all_graphs, outputs=[graph1, graph2, graph3, graph4, graph5, all_graph])

    with gr.Tab("Summary Cell"):
        btn_summary = gr.Button("Show Summary Table")
        gr.Markdown("### VQE Results Summary Table")
        summary_md = gr.Markdown()

        def show_summary():
            rows = globals().get('summary_rows', [])
            if rows:
                table = get_summary_table(rows)
            else:
                table = "No summary available. Run summary cell in notebook."
            for partial in live_text_reveal(table):
                yield partial
        btn_summary.click(show_summary, outputs=summary_md)

    with gr.Tab("Extras"):
        btn_notes = gr.Button("Save Note")
        gr.Markdown("### Add Notes or Observations")
        notes = gr.Textbox(label="Your Notes", lines=6, interactive=True)
        notes_display = gr.Markdown()

        def save_note(note):
            return f"**Your Note:**\n\n{note}"
        btn_notes.click(save_note, inputs=notes, outputs=notes_display)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4743e6296621e17942.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
